# [CoE202] **[Homework3]** CNN Classification for CIFAR10 (Pytorch)

## **Testing Script**

In this section, you are going to **test** CNN classification for CIFAR10 in the Pytorch framework.


## 0. Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt # this is for importing matplotlib.pyplot (library for graph plot)
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##1. Define your model to load weights, and define your transfomation in Dataloader

In [5]:
#You can only change this cell

#define your model
class CIFAR_Net(nn.Module): 
    def __init__(self):
        super(CIFAR_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.conv1 = nn.Conv2d(3,32,3,1)       #nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.conv2 = nn.Conv2d(32,64,3,1) 
        self.conv3 = nn.Conv2d(64,128,3,1)      
        #self.conv4 = nn.Conv2d(128,128,3,1)  
        self.fc1 = nn.Linear(4608,1000)         #nn.Linear(in_features, out_features)
        self.fc2 = nn.Linear(1000,200) 
        self.fc3 = nn.Linear(200,10)  
        self.bat1 = nn.BatchNorm2d(32)
        self.bat2 = nn.BatchNorm2d(64)
        self.bat3 = nn.BatchNorm2d(128)
        self.dropout = nn.Dropout(p=0.5, inplace=False)
        self.maxpool = nn.MaxPool2d(2,2)  #nn.MaxPool2d(kernel_size, stride)
        self.relu = nn.ReLU()

    def forward(self, x):
        # size of input is 3 x 32 x 32
        x = self.conv1(x) # conv1
        x = self.bat1(x)
        x = self.relu(x) # relu
        #x = self.dropout(x)
        x = self.conv2(x) # conv2
        x = self.bat2(x)
        x = self.relu(x) # relu
       # x = self.dropout(x)
        x = self.maxpool(x) # maxpool  
        x = self.conv3(x) # conv2
        x = self.bat3(x)
        x = self.relu(x) # relu
        #x = self.dropout(x)
        #x = self.conv4(x) # conv4
        ##x = self.bat3(x)
        #x = self.relu(x) # relu
        #x = self.dropout(x)
        x = self.maxpool(x) # maxpool   

        x = x.view(-1, 4608)  # view=flatten (what is the size of x at this point?)

        x = self.fc1(x) # fc1
        x = self.relu(x) # relu
        x = self.fc2(x) # fc2
        x = self.relu(x) # relu
        x = self.fc3(x) # fc3
        return x

#define your transformation on input image, groundtruth
transform = transforms.Compose([
    transforms.ToTensor(),
])
target_transform = None


## 2. Load test dataset

In [6]:
test_dataset = torchvision.datasets.CIFAR10("../cifar10_data/",
                                              download=True,
                                              train=False,
                                              transform=transform,
                                              target_transform=target_transform)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle=False, drop_last = True)



Files already downloaded and verified


##3. Load your model weights

In [7]:
model_save_name = 'checkpoint.pth'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model = CIFAR_Net()
model.load_state_dict(torch.load(path))
model.cuda()
model.eval()

CIFAR_Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4608, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
  (bat1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bat2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bat3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
)

##4. Check model accuracy

In [8]:
def thresholding(prediction):
    """Find label which shows highest prediction value.
    
    Args:
        prediction (torch.tensor) : prediction tensor. Dimension of [Batch size, 10]
    
    Returns:
        pred_label (torch.tensor) : Index which shows maximum prediction value. Dimension of [Batch size]
    """
    _, pred_label = torch.max(prediction, 1)
    return pred_label

In [9]:
model.eval()
total_cnt, correct_cnt = 0.0, 0.0
for batch_idx, (x, target) in enumerate(test_loader):
    with torch.no_grad():
      x, target = x.cuda(), target.cuda()
      prediction = model(x)
      total_cnt += x.data.size(0)
      correct_cnt += (thresholding(prediction) == target.data).sum().item()

print(f"Test Accuracy = {100*correct_cnt/total_cnt}%")

Test Accuracy = 81.81%
